### Fine-tunning with VGG16 convolutional base

In [25]:
# pretrain the FC layer then to fine-tunning the vgg16 conv_base net
import os
import Utils
from keras import models
from keras import layers
from keras.applications.vgg16 import VGG16

conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(48, 48, 3))

model = models.Sequential()
# convert one channel to three without preprocess the grayscale to colored image.
model.add(layers.Conv2D(3, kernel_size=(1, 1), input_shape=(48, 48, 1), activation='relu'))
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))
conv_base.trainable = False  # fixed the convolution base before fine-tunning the end-to-end network
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 48, 48, 3)         6         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 7175      
Total params: 15,247,181
Trainable params: 532,493
Non-trainable params: 14,714,688
__________________________________________________________

In [26]:
from keras import optimizers

train_generator = Utils.image_data_generator(Utils.train_dir)
validation_generator = Utils.image_data_generator(Utils.validation_dir)

model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.RMSprop(lr=1e-4, decay=1e-6),
            metrics=['acc'])

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


####  note: before train 20 epoch, there is 70 epoch pre-train with the same net architeture, but the model seem still underfitting.

In [27]:
history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=1400,
                            epochs=80,
                            validation_data=validation_generator,
                            validation_steps=179)

Epoch 1/80
1400/1400 [==============================] - 73s 52ms/step - loss: 1.7378 - acc: 0.2960 - val_loss: 1.6364 - val_acc: 0.3662
Epoch 2/80
1400/1400 [==============================] - 73s 52ms/step - loss: 1.6484 - acc: 0.3495 - val_loss: 1.6079 - val_acc: 0.3779
Epoch 3/80
1400/1400 [==============================] - 73s 52ms/step - loss: 1.6149 - acc: 0.3681 - val_loss: 1.5920 - val_acc: 0.3860
Epoch 4/80
1400/1400 [==============================] - 73s 52ms/step - loss: 1.5970 - acc: 0.3768 - val_loss: 1.5803 - val_acc: 0.3927
Epoch 5/80
1400/1400 [==============================] - 74s 53ms/step - loss: 1.5781 - acc: 0.3863 - val_loss: 1.5768 - val_acc: 0.3902
Epoch 6/80
1400/1400 [==============================] - 73s 52ms/step - loss: 1.5647 - acc: 0.3946 - val_loss: 1.5659 - val_acc: 0.3933
Epoch 7/80
1400/1400 [==============================] - 74s 53ms/step - loss: 1.5581 - acc: 0.3988 - val_loss: 1.5673 - val_acc: 0.3941
Epoch 8/80
1400/1400 [==========================

In [28]:
score = Utils.evaluate_model(model)
print('end_to_end_before_fine_tunning_with_VGG16_pretrain_net_hit_%2.2f%%' % (score[1] * 100))

Found 3589 images belonging to 7 classes.
end_to_end_before_fine_tunning_with_VGG16_pretrain_net_hit_44.61%


In [21]:
Utils.save_model(model, 'end_to_end_before_fine_tunning_vgg16_70epochs_hit(48.23%).h5')

In [5]:
Utils.plt_acc_loss(history, 'end2end_before_fine_tunning_vgg16_accuracy', 'end2end_before_fine_tunning_vgg16_loss')

NameError: name 'history' is not defined

In [29]:
# note: you can directely train the model rather than load a better before fine tuning model
# here it's just to demostrate the process of fine-tuning VGG16.
import Utils
model = Utils.load_model('end_to_end_before_fine_tunning_vgg16hit(50.77%).h5')
model.summary()
conv_base = model.layers[1]
# for layer in conv_base.layers:
#   print(layer.name, layer.trainable)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 3)         6         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 7)                 7175      
Total params: 15,247,181
Trainable params: 7,611,917
Non-trainable params: 7,635,264
_________________________________________________________

L:\Anaconda3\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [18]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    layer.trainable = set_trainable
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 3)         6         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 7)                 7175      
Total params: 15,247,181
Trainable params: 5,252,109
Non-trainable params: 9,995,072
_________________________________________________________

In [30]:
from keras import optimizers

train_generator = Utils.image_data_generator(Utils.train_dir)
validation_generator = Utils.image_data_generator(Utils.validation_dir)

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4, decay=1e-6),
             metrics=['acc'])


hist = model.fit_generator(train_generator,
                           steps_per_epoch=1400,
                           epochs=20,
                           validation_data=validation_generator,
                           validation_steps=179)                           

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/20
1400/1400 [==============================] - 87s 62ms/step - loss: 2.2774 - acc: 0.4385 - val_loss: 1.4274 - val_acc: 0.4656
Epoch 2/20
1400/1400 [==============================] - 86s 61ms/step - loss: 1.2450 - acc: 0.5428 - val_loss: 1.3506 - val_acc: 0.5151
Epoch 3/20
1400/1400 [==============================] - 86s 61ms/step - loss: 1.1139 - acc: 0.5973 - val_loss: 1.4104 - val_acc: 0.5209
Epoch 4/20
1400/1400 [==============================] - 86s 61ms/step - loss: 1.0089 - acc: 0.6407 - val_loss: 1.5631 - val_acc: 0.4796
Epoch 5/20
1400/1400 [==============================] - 86s 62ms/step - loss: 0.9240 - acc: 0.6792 - val_loss: 1.7178 - val_acc: 0.5232
Epoch 6/20
1400/1400 [==============================] - 86s 62ms/step - loss: 0.8342 - acc: 0.7138 - val_loss: 1.7121 - val_acc: 0.5229
Epoch 7/20
1400/1400 [==============================] - 86s 62ms/step - loss: 0.7651 - acc: 0.7420 

In [9]:
score = Utils.evaluate_model(model)
print('svm on the top of pretrain VGG16 hit %.2f%% accuracy' % (score[1] * 100))

Found 3589 images belonging to 7 classes.
svm on the top of pretrain VGG16 hit 55.06% accuracy


In [10]:
Utils.save_model(model, 'fine-tuning VGG16(100epoch_before_hit46.89%) 30epoch hit %.2f%%.h5' % (score[1] * 100))

ValueError: unsupported format character ')' (0x29) at index 43